In [1]:
import numpy as np
import osmnx as ox
from osgeo import osr, ogr

In [160]:
place_name = 'Пермь Индустриальный район'
geo_df = ox.footprints_from_place(place_name)

geo_df = geo_df[['building', 'building:levels', 'geometry']]
geo_df = geo_df.reset_index(drop=True)

geo_df["area"], geo_df["edges_count"] = None, None
geo_df.loc[geo_df['building:levels'].isnull(), 'building:levels'] = 1
geo_df

,building,building:levels,geometry,area,edges_count
0,yes,2,"POLYGON ((56.14878 57.97648, 56.14874 57.97621...",None,None
1,yes,2,"POLYGON ((56.14644 57.97661, 56.14662 57.97641...",None,None
2,yes,2,"POLYGON ((56.14640 57.97590, 56.14658 57.97571...",None,None
3,yes,3,"POLYGON ((56.18953 57.97822, 56.19003 57.97771...",None,None
4,yes,2,"POLYGON ((56.18588 57.97636, 56.18575 57.97636...",None,None
...,...,...,...,...,...
3492,yes,1,"POLYGON ((56.17454 57.87818, 56.17385 57.87859...",None,None
3493,yes,1,"POLYGON ((56.17115 57.88024, 56.17135 57.88034...",None,None
3494,yes,1,"POLYGON ((56.15870 57.88619, 56.15877 57.88621...",None,None
3495,yes,1,"POLYGON ((56.17272 57.87929, 56.17281 57.87933...",None,None


In [161]:
for index, element in geo_df.iterrows():
    geo_df["area"][index] = get_area_from_polygon(element.geometry)
    geo_df["edges_count"][index] = element.geometry.exterior.coords.__len__()

<ipython-input-161-24d48abced48>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_df["area"][index] = get_area_from_polygon(element.geometry)
<ipython-input-161-24d48abced48>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geo_df["edges_count"][index] = element.geometry.exterior.coords.__len__()


In [162]:
geo_df

,building,building:levels,geometry,area,edges_count
0,yes,2,"POLYGON ((56.14878 57.97648, 56.14874 57.97621...",1130.43,9
1,yes,2,"POLYGON ((56.14644 57.97661, 56.14662 57.97641...",1407.07,9
2,yes,2,"POLYGON ((56.14640 57.97590, 56.14658 57.97571...",1527.28,14
3,yes,3,"POLYGON ((56.18953 57.97822, 56.19003 57.97771...",14304.9,30
4,yes,2,"POLYGON ((56.18588 57.97636, 56.18575 57.97636...",11899.3,31
...,...,...,...,...,...
3492,yes,1,"POLYGON ((56.17454 57.87818, 56.17385 57.87859...",1213.44,5
3493,yes,1,"POLYGON ((56.17115 57.88024, 56.17135 57.88034...",731.197,5
3494,yes,1,"POLYGON ((56.15870 57.88619, 56.15877 57.88621...",1278.19,24
3495,yes,1,"POLYGON ((56.17272 57.87929, 56.17281 57.87933...",1319.12,7


In [163]:
geo_df = geo_df.drop(['geometry'], axis=1)
geo_df = geo_df.reset_index(drop=True)

geo_df.edges_count = geo_df.edges_count.astype('float')
geo_df = geo_df[np.isfinite(geo_df['edges_count'])]
geo_df = geo_df.fillna(0)

living_buildings = ['apartments', 'bungalow', 'cabin', 'detached', 'dormitory', 'farm', 'ger', 'hotel',
                           'house', 'houseboat', 'residential', 'semidetached_house', 'static_caravan', 'terrace']
real_live_buildings = geo_df.loc[geo_df.building.isin(living_buildings)]
living_building_dict = dict(real_live_buildings.building.value_counts())

In [164]:
population, fake_houses_count = calculate_people_count(geo_df)
print(f"В {place_name} проживает {round(population)} человек\n"
      f"Разновидности жилых зданий в {place_name}: {living_building_dict}, их количество {sum(living_building_dict.values())}\n"
      f"Предполагаемых жилых зданий в {place_name}: {fake_houses_count}")

В Пермь Индустриальный район проживает 146350 человек
Разновидности жилых зданий в Пермь Индустриальный район: {'apartments': 640, 'house': 57, 'residential': 6, 'dormitory': 6, 'hotel': 2, 'detached': 1}, их количество 712
Предполагаемых жилых зданий в Пермь Индустриальный район: 483


In [165]:
def get_area_from_polygon(geometry_obj):
    # TODO: сделать для мультиполигона тоже
    json = {'type': 'Polygon',
            'coordinates': [list(geometry_obj.exterior.coords)]}

    source = osr.SpatialReference()
    source.ImportFromEPSG(4326)
    target = osr.SpatialReference()
    target.ImportFromEPSG(5243)

    transform = osr.CoordinateTransformation(source, target)
    poly = ogr.CreateGeometryFromJson(str(json).replace('(', '[').replace(')', ']'))
    poly.Transform(transform)
    area = poly.GetArea()
    return area

def calculate_people_count(geo_df):
    people_count, fake_houses = 0, 0
    for index, row in geo_df.iterrows():
        if row['building'] in living_buildings:
            people_count += row.area * int(row['building:levels']) / 2 / 22
        elif int(row['building:levels']) > 1 and row['building'] == 'yes' and row['edges_count'] < 13:
            people_count += row.area * int(row['building:levels']) / 2 / 22
            fake_houses += 1
    return people_count, fake_houses

In [ ]:
place_name = 'Пермь Свердловский район'
dateea = ox.footprints_from_place(place_for_search)

In [95]:
dateea

,nodes,addr:housenumber,addr:housenumber2,addr:street,addr:street2,building,building:levels,geometry,amenity,architect,...,building:material,abandoned,diaper,toilets:disposal,unisex,construction,preschool,members,type,aeroway
24223415,"[262279469, 262279470, 262279471, 1601214377, ...",113,23,Уральская улица,улица Крупской,apartments,5,"POLYGON ((56.27796 58.01913, 56.27816 58.01919...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24268090,"[262749035, 262749036, 262749037, 262749038, 2...",25,NaN,улица Крупской,NaN,apartments,5,"POLYGON ((56.27800 58.01906, 56.27864 58.01835...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24268098,"[262750118, 262750119, 262750120, 262750121, 2...",27,48,улица Крупской,улица Лебедева,apartments,5,"POLYGON ((56.27871 58.01828, 56.27899 58.01796...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24268138,"[262750938, 262750939, 262750940, 262750941, 2...",46,NaN,улица Лебедева,NaN,yes,5,"POLYGON ((56.28031 58.01830, 56.28141 58.01857...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24268141,"[262751605, 262751606, 262751607, 262751608, 2...",46А,NaN,улица Лебедева,NaN,kindergarten,2,"POLYGON ((56.27956 58.01860, 56.28003 58.01871...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3421905,NaN,NaN,NaN,NaN,NaN,yes,NaN,"POLYGON ((56.25800 57.98074, 56.25799 57.98055...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{167565569: 'outer', 255534998: 'outer'}",multipolygon,NaN
3421908,NaN,NaN,NaN,NaN,NaN,industrial,NaN,"POLYGON ((56.25362 57.96869, 56.25238 57.96872...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{167565573: 'outer', 255534993: 'outer'}",multipolygon,NaN
3436636,NaN,NaN,NaN,NaN,NaN,yes,NaN,"POLYGON ((56.24230 57.99971, 56.24244 57.99975...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{256432858: 'outer'},multipolygon,hangar
4817206,NaN,NaN,NaN,NaN,NaN,yes,2,"POLYGON ((56.27463 57.96646, 56.27421 57.96685...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{339741813: 'outer', 106475983: 'outer'}",multipolygon,NaN


In [111]:
print(dateea['geometry'][6101546].type == 'MultiPolygon')
(dateea['geometry'][4817206].type) == 'Polygon'

True


True